This notebook has for goal to preprocess the Kaggle Olympic 2024 dataset. 

The original dataset can be found [here](https://www.kaggle.com/datasets/piterfm/paris-2024-olympic-summer-games)

# Description of the dataset before processing

The Kaggle Olympic 2024 (KO) is a dataset containing the description of all athletes who participated in the Olympic games of 2024. It is already well curated for most general tasks (~~but this project is anything but general~~) but will require some additional curation given our usage of the names. 

With 11114 entries, it is our biggest dataset. 

# Preprocessing tasks

Firstly, the services that can make use of the country of origin of the name, for the most part, only accept the ISO code of the country. Since the country is only registered as a string here, we use pycountry to make a fuzzy search of the string and return the ISO code. 

Secondly, we'll check for duplicate on the basis of the full name and the country, and remove them. 

Then, names that are considered difficult needs to be flagged. Names are considered difficult if they contains any of the following : 
- Middle name
- No last name
- Double barreled names (ex : Smith-Jones)
- ASCII characters (any character that is not the standard a-zA-Z. All accents are included here too.)
- Numbers in the name
- trailing spaces and other irregularities with blank spaces. 
- Potential punctuation in the name (ex: Adam J.)

A visual inspection of a part of the dataset seems to allude to the fact that it follows a structure for the full name. The last name is written in all upper caps, while the first name has only the first letter capitalized. This would mean that, at least for name that NOT flagged earlier, separating them into first and last name can be done via code. Going forward, this is assumed to be true. 

In [75]:
import numpy as np
import pandas as pd
from pathlib import Path
import pycountry

In [76]:
df = pd.read_csv('../../data/rawData/kaggleOlympic2024/KaggleOlympic2024.csv')

In [77]:
df.head()

,name,name_short,name_tv,gender,function,country_code,country,country_long,nationality,nationality_long,nationality_code,birth_date,birth_place,birth_country,residence_place,residence_country,lang
0,ALEKSANYAN Artur,ALEKSANYAN A,Artur ALEKSANYAN,Male,Athlete,ARM,Armenia,Armenia,Armenia,Armenia,ARM,21/10/1991,GYUMRI,Armenia,GYUMRI,Armenia,"Armenian, English, Russian"
1,AMOYAN Malkhas,AMOYAN M,Malkhas AMOYAN,Male,Athlete,ARM,Armenia,Armenia,Armenia,Armenia,ARM,22/01/1999,YEREVAN,Armenia,YEREVAN,Armenia,Armenian
2,GALSTYAN Slavik,GALSTYAN S,Slavik GALSTYAN,Male,Athlete,ARM,Armenia,Armenia,Armenia,Armenia,ARM,21/12/1996,NaN,NaN,YEREVAN,Armenia,Armenian
3,HARUTYUNYAN Arsen,HARUTYUNYAN A,Arsen HARUTYUNYAN,Male,Athlete,ARM,Armenia,Armenia,Armenia,Armenia,ARM,22/11/1999,MASIS,Armenia,YEREVAN,Armenia,Armenian
4,TEVANYAN Vazgen,TEVANYAN V,Vazgen TEVANYAN,Male,Athlete,ARM,Armenia,Armenia,Armenia,Armenia,ARM,27/10/1999,POKR VEDI,Armenia,NaN,Armenia,"Armenian, Russian"


## Adding ISO code

The dataset already provide an ISO code for the country and nationality of the participants, but it is in another ISO standard (ISO 3166-1 alpha 3, where as the services overwhelmingly used ISO 3166-1 alpha 2). Additionaly, this ISO code is set to indicate the country's team the athlete is playing for, not their own birth country, which is likely to carry a stronger influence on how they are named. 

As such, the procedure is as follow : 
1. We use pycountry to fuzzy search the 'birth_country' columns to return a matching ISO code in the correct standard
2. If this fails, we then use the nationality then country of the athlete to define the ISO alpha 2 code. 
3. If this fails, either because the values are missing or the fuzzy search fails, we use the 'country_code' to still fill in a valid ISO code. 

After this, some error and inconcistencies where noticed in the ISO code used in the dataset. For example, the code noted for the Islamic Republic of Iran was 'IRI', when the real code (according to Wikipedia as of 04/09/2025) is 'IRN'

In [78]:
def findISOCode(row):
    candidates = [row.get("birth_country"), row.get('nationality'), row.get('country')]

    for birth_country in candidates:
        if isinstance(birth_country, str) and birth_country.strip():
            try:
                return pycountry.countries.search_fuzzy(birth_country)[0].alpha_2
            except LookupError:
                pass
    else : 
        try : 
            fallBack = row.get("country_code")
            return pycountry.countries.get(alpha_3=fallBack).alpha_2
        except AttributeError:
            
            return None

df["iso_country"] = df.apply(findISOCode, axis=1)
df.head()

,name,name_short,name_tv,gender,function,country_code,country,country_long,nationality,nationality_long,nationality_code,birth_date,birth_place,birth_country,residence_place,residence_country,lang,iso_country
0,ALEKSANYAN Artur,ALEKSANYAN A,Artur ALEKSANYAN,Male,Athlete,ARM,Armenia,Armenia,Armenia,Armenia,ARM,21/10/1991,GYUMRI,Armenia,GYUMRI,Armenia,"Armenian, English, Russian",AM
1,AMOYAN Malkhas,AMOYAN M,Malkhas AMOYAN,Male,Athlete,ARM,Armenia,Armenia,Armenia,Armenia,ARM,22/01/1999,YEREVAN,Armenia,YEREVAN,Armenia,Armenian,AM
2,GALSTYAN Slavik,GALSTYAN S,Slavik GALSTYAN,Male,Athlete,ARM,Armenia,Armenia,Armenia,Armenia,ARM,21/12/1996,NaN,NaN,YEREVAN,Armenia,Armenian,AM
3,HARUTYUNYAN Arsen,HARUTYUNYAN A,Arsen HARUTYUNYAN,Male,Athlete,ARM,Armenia,Armenia,Armenia,Armenia,ARM,22/11/1999,MASIS,Armenia,YEREVAN,Armenia,Armenian,AM
4,TEVANYAN Vazgen,TEVANYAN V,Vazgen TEVANYAN,Male,Athlete,ARM,Armenia,Armenia,Armenia,Armenia,ARM,27/10/1999,POKR VEDI,Armenia,NaN,Armenia,"Armenian, Russian",AM


In [79]:
na_isoCode = df['iso_country'].isna().sum()
print(f"The dataset has {na_isoCode} missing iso codes")

df_na_isoCode = df[df['iso_country'].isna()]
df_na_isoCode

The dataset has 113 missing iso codes


,name,name_short,name_tv,gender,function,country_code,country,country_long,nationality,nationality_long,nationality_code,birth_date,birth_place,birth_country,residence_place,residence_country,lang,iso_country
77,AL SAYEGH Safia,AL SAYEGH S,Safia AL SAYEGH,Female,Athlete,UAE,UA Emirates,United Arab Emirates,UA Emirates,United Arab Emirates,UAE,23/09/2001,NaN,NaN,NaN,NaN,"Arabic, English",None
98,ABDOLI Samyar,ABDOLI S,Samyar ABDOLI,Male,Athlete,IRI,IR Iran,Islamic Republic of Iran,IR Iran,Islamic Republic of Iran,IRI,25/12/2002,NaN,NaN,NaN,NaN,NaN,None
100,AL MARZOOQI Omar Abdul Aziz,AL MARZOOQI OAA,Omar Abdul Aziz AL MARZOOQI,Male,Athlete,UAE,UA Emirates,United Arab Emirates,UA Emirates,United Arab Emirates,UAE,28/03/2003,NaN,NaN,ABU DHABI,UA Emirates,"Arabic, English",None
125,SALIMI Arian,SALIMI A,Arian SALIMI,Male,Athlete,IRI,IR Iran,Islamic Republic of Iran,IR Iran,Islamic Republic of Iran,IRI,16/12/2003,KERMANSHAH,IR Iran,NaN,IR Iran,Farsi,None
126,BARKHORDARI Mehran,BARKHORDARI M,Mehran BARKHORDARI,Male,Athlete,IRI,IR Iran,Islamic Republic of Iran,IR Iran,Islamic Republic of Iran,IRI,26/07/2000,QAZVIN,IR Iran,NaN,IR Iran,Farsi,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9545,ROBAN Handal,ROBAN H,Handal ROBAN,Male,Athlete,VIN,StVincent&Grenadines,St Vincent and the Grenadines,StVincent&Grenadines,St Vincent and the Grenadines,VIN,05/09/2002,NaN,StVincent&Grenadines,"UNIVERSITY PARK, PA",United States,English,None
9546,MALONEY Shafiqua,MALONEY S,Shafiqua MALONEY,Female,Athlete,VIN,StVincent&Grenadines,St Vincent and the Grenadines,StVincent&Grenadines,St Vincent and the Grenadines,VIN,27/02/1999,RICHLAND PARK,StVincent&Grenadines,"FAYETTEVILLE, AK",United States,"English, Spanish",None
10990,ALSHEHHI Maha,ALSHEHHI M,Maha ALSHEHHI,Female,Athlete,UAE,UA Emirates,United Arab Emirates,UA Emirates,United Arab Emirates,UAE,12/07/2006,NaN,NaN,NaN,NaN,NaN,None
10991,ALFARSI Mariam,ALFARSI M,Mariam ALFARSI,Female,Athlete,UAE,UA Emirates,United Arab Emirates,UA Emirates,United Arab Emirates,UAE,15/01/2008,NaN,NaN,DUBAI,UA Emirates,"Arabic, English",None


In [80]:
df_na_isoCode['birth_country'].value_counts()

birth_country
Chinese Taipei          50
IR Iran                 32
StVincent&Grenadines     2
UA Emirates              1
Name: count, dtype: int64

In [81]:
na_isoCode_dict = { # Those codes are based on the wikipedia articles on ISO 3166-1 alpha-2
    'Chinese Taipei' : 'CN',
    'IR Iran' : 'IR',
    'UA Emirates' : 'AE',
    'StVincent&Grenadines':'VC',
    'St Kitts and Nevis' : 'KN'
}
df.loc[df['iso_country'].isna(), 'iso_country'] = (
    df.loc[df['iso_country'].isna(), 'birth_country']# use the birth country first
      .map(na_isoCode_dict)
      .fillna(df.loc[df['iso_country'].isna(), 'iso_country'])  
)
df.loc[df['iso_country'].isna(), 'iso_country'] = (
    df.loc[df['iso_country'].isna(), 'country'] # Use the country next. 
      .map(na_isoCode_dict)
      .fillna(df.loc[df['iso_country'].isna(), 'iso_country'])  
)

In [82]:
na_isoCode = df['iso_country'].isna().sum()
print(f"The dataset has {na_isoCode} missing iso codes")

The dataset has 0 missing iso codes


## Flagging names

In [83]:
middle_name_mask = df['name'].str.strip().str.count(" ") >= 2

middle_name_count = middle_name_mask.sum()

print(f"Total rows: {df.shape[0]}")
print(f"Potential middle names: {middle_name_count}")
print(f"Percentage: {middle_name_count / df.shape[0] * 100:.2f}%")

df['hasMiddleName'] = middle_name_mask

Total rows: 11113
Potential middle names: 1682
Percentage: 15.14%


In [84]:
no_last_name_mask = df['name'].str.strip().str.count(" ") == 0
no_last_name_count = no_last_name_mask.sum()

print(f"Total rows: {df.shape[0]}")
print(f"Potential rows without last name: {no_last_name_count}")
print(f"Percentage: {no_last_name_count / df.shape[0] * 100:.2f}%")

df['hasNoLastName'] = no_last_name_mask

Total rows: 11113
Potential rows without last name: 54
Percentage: 0.49%


In [85]:
import re
import unicodedata
# This code was generated by ChatGPT and tweaked by me. 

# 1) Keep the original text; add a normalized helper column for consistent tests
def normalize_name(s):
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKC", str(s))  # unify apostrophes/spaces, etc.
    s = s.strip()
    s = re.sub(r"\s+", " ", s)                # collapse internal whitespace
    return s

df["person_label_norm"] = df["name"].map(normalize_name)

# 2) Build masks (vectorized)
s_raw  = df["name"].astype(str)
s_norm = df["person_label_norm"].astype(str)

hyphen_mask       = s_norm.str.contains(r"-", na=False)
apostrophe_mask   = s_norm.str.contains(r"['\u2019\u02BC]", na=False)  # ', ’, ʼ
punct_mask        = s_norm.str.contains(r"[.,/&(){}\[\]<>@#?!$%^*_=+\\]", na=False)
digit_mask        = s_norm.str.contains(r"\d", na=False)
initial_mask      = s_norm.str.contains(r"(^|\s)[A-Za-z]\.", na=False)  # initials like "J."
# Non-ASCII: after NFKC, compare ASCII-stripped version to itself
non_ascii_mask    = ~s_norm.map(lambda x: x.isascii())

# Whitespace issues measured on the raw (pre-normalization) text
whitespace_mask   = s_raw.str.contains(
    r"^\s|[\u00A0\u2007\u202F]|\s{2,}|\s$", na=False
)

# 3) Combine into a single "hasDifficultName" flag
difficult_mask = (
    middle_name_mask
    | no_last_name_mask
    | hyphen_mask
    | apostrophe_mask
    | punct_mask
    | digit_mask
    | initial_mask
    | non_ascii_mask
    | whitespace_mask
)

df["hasDifficultName"] = difficult_mask

# 4) (Optional) Keep a reason code for explainability/auditing
def reason_row(i):
    reasons = []
    if middle_name_mask.iat[i]: reasons.append("middleName")
    if no_last_name_mask.iat[i]: reasons.append("noLastName")
    if hyphen_mask.iat[i]:      reasons.append("hyphen")
    if apostrophe_mask.iat[i]:  reasons.append("apostrophe")
    if punct_mask.iat[i]:       reasons.append("punctuation")
    if digit_mask.iat[i]:       reasons.append("digit")
    if initial_mask.iat[i]:     reasons.append("initials")
    if non_ascii_mask.iat[i]:   reasons.append("non_ascii")
    if whitespace_mask.iat[i]:  reasons.append("whitespace")
    return "|".join(reasons)

df["difficult_reason"] = [reason_row(i) for i in range(len(df))]

# 5) (Optional) Quick summary for your report
summary = {
    "total": len(df),
    "difficult_count": difficult_mask.sum(),
    "difficult_pct": 100 * difficult_mask.mean(),
    "by_reason": {
        "hasMiddleName" : int(middle_name_mask.sum()),
        "hasNoLastName" : int(no_last_name_mask.sum()),
        "hyphen": int(hyphen_mask.sum()),
        "apostrophe": int(apostrophe_mask.sum()),
        "punctuation": int(punct_mask.sum()),
        "digit": int(digit_mask.sum()),
        "initials": int(initial_mask.sum()),
        "non_ascii": int(non_ascii_mask.sum()),
        "whitespace": int(whitespace_mask.sum()),
    },
}
print(summary)

C:\Users\brief\AppData\Local\Temp\ipykernel_24276\1200723681.py:24: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  initial_mask      = s_norm.str.contains(r"(^|\s)[A-Za-z]\.", na=False)  # initials like "J."


{'total': 11113, 'difficult_count': np.int64(2031), 'difficult_pct': np.float64(18.27589309817331), 'by_reason': {'hasMiddleName': 1682, 'hasNoLastName': 54, 'hyphen': 279, 'apostrophe': 39, 'punctuation': 11, 'digit': 2, 'initials': 8, 'non_ascii': 0, 'whitespace': 0}}


## Separating non-flagged names

In [86]:
def split_name(full_name: str):
    tokens = full_name.strip().split()
    
    return tokens[0], tokens[1]

# Apply to dataset
df[["lastName", "surName"]] = df[df['hasDifficultName']==False]["name"].apply(
    lambda x: pd.Series(split_name(x))
)

df.head(50)

,name,name_short,name_tv,gender,function,country_code,country,country_long,nationality,nationality_long,...,residence_country,lang,iso_country,hasMiddleName,hasNoLastName,person_label_norm,hasDifficultName,difficult_reason,lastName,surName
0,ALEKSANYAN Artur,ALEKSANYAN A,Artur ALEKSANYAN,Male,Athlete,ARM,Armenia,Armenia,Armenia,Armenia,...,Armenia,"Armenian, English, Russian",AM,False,False,ALEKSANYAN Artur,False,,ALEKSANYAN,Artur
1,AMOYAN Malkhas,AMOYAN M,Malkhas AMOYAN,Male,Athlete,ARM,Armenia,Armenia,Armenia,Armenia,...,Armenia,Armenian,AM,False,False,AMOYAN Malkhas,False,,AMOYAN,Malkhas
2,GALSTYAN Slavik,GALSTYAN S,Slavik GALSTYAN,Male,Athlete,ARM,Armenia,Armenia,Armenia,Armenia,...,Armenia,Armenian,AM,False,False,GALSTYAN Slavik,False,,GALSTYAN,Slavik
3,HARUTYUNYAN Arsen,HARUTYUNYAN A,Arsen HARUTYUNYAN,Male,Athlete,ARM,Armenia,Armenia,Armenia,Armenia,...,Armenia,Armenian,AM,False,False,HARUTYUNYAN Arsen,False,,HARUTYUNYAN,Arsen
4,TEVANYAN Vazgen,TEVANYAN V,Vazgen TEVANYAN,Male,Athlete,ARM,Armenia,Armenia,Armenia,Armenia,...,Armenia,"Armenian, Russian",AM,False,False,TEVANYAN Vazgen,False,,TEVANYAN,Vazgen
5,ARENAS Lorena,ARENAS L,Lorena ARENAS,Female,Athlete,COL,Colombia,Colombia,Colombia,Colombia,...,Australia,Spanish,CO,False,False,ARENAS Lorena,False,,ARENAS,Lorena
6,McKENZIE Ashley,McKENZIE A,Ashley McKENZIE,Male,Athlete,JAM,Jamaica,Jamaica,Jamaica,Jamaica,...,Great Britain,English,GB,False,False,McKENZIE Ashley,False,,McKENZIE,Ashley
7,BASS BITTAYE Gina Mariam,BASS BITTAYE GM,Gina Mariam BASS BITTAYE,Female,Athlete,GAM,Gambia,Gambia,Gambia,Gambia,...,NaN,"English, French",GM,True,False,BASS BITTAYE Gina Mariam,True,middleName,NaN,NaN
8,CAMARA Ebrahima,CAMARA E,Ebrahima CAMARA,Male,Athlete,GAM,Gambia,Gambia,Gambia,Gambia,...,France,"Arabic, English, French, Mandinka, Wolof",GM,False,False,CAMARA Ebrahima,False,,CAMARA,Ebrahima
9,RUEDA SANTOS Lizeth,RUEDA SANTOS L,Lizeth RUEDA SANTOS,Female,Athlete,MEX,Mexico,Mexico,Mexico,Mexico,...,Mexico,Spanish,MX,True,False,RUEDA SANTOS Lizeth,True,middleName,NaN,NaN


In [87]:
na_surName = df['surName'].isna().sum() # Checking if no name were separated incorrectly. This should output the same number for both sur and last name. 
na_lastName = df['lastName'].isna().sum()
print(f"The dataset has {na_lastName} missing last name, and {na_surName} missing surname")

The dataset has 2031 missing last name, and 2031 missing surname


# Recording preprocessed dataset

In [88]:
df.to_csv('../../data/KaggleOlympic2024.csv', index=False)

In [89]:
check = pd.read_csv('../../data/KaggleOlympic2024.csv')
check.shape

(11113, 25)

In [90]:
check.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11113 entries, 0 to 11112
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               11113 non-null  object
 1   name_short         11110 non-null  object
 2   name_tv            11110 non-null  object
 3   gender             11113 non-null  object
 4   function           11113 non-null  object
 5   country_code       11113 non-null  object
 6   country            11113 non-null  object
 7   country_long       11113 non-null  object
 8   nationality        11110 non-null  object
 9   nationality_long   11110 non-null  object
 10  nationality_code   11110 non-null  object
 11  birth_date         11113 non-null  object
 12  birth_place        8727 non-null   object
 13  birth_country      9475 non-null   object
 14  residence_place    6804 non-null   object
 15  residence_country  8288 non-null   object
 16  lang               10605 non-null  objec

In [91]:
df.isna().sum().sort_values(ascending=False)

residence_place      4309
residence_country    2825
birth_place          2386
lastName             2031
surName              2031
birth_country        1638
lang                  508
nationality             3
name_short              3
nationality_code        3
nationality_long        3
name_tv                 3
country                 0
gender                  0
name                    0
country_code            0
function                0
country_long            0
birth_date              0
iso_country             0
hasMiddleName           0
person_label_norm       0
hasNoLastName           0
difficult_reason        0
hasDifficultName        0
dtype: int64